# 🚀 Quickstart

In this quickstart guide, we describe how to use `drlearn` to solve distributionally robust optimization (DRO) problems of the form

$$
    \min_{w \in \mathbb{R}^d} \max_{q \in \mathcal{Q}} q^\top \ell(w) - \nu D(q \Vert \mathbf{1}/n)
$$
where:
- $w$ denotes the parameters of a model (the "primal variables"),
- $q$ denotes the weights on individual training examples (the "dual variables"),
- $\ell: \mathbb{R}^d \rightarrow \mathbb{R}^n$ denotes a loss function for individual training examples,
- $D(\cdot \Vert \mathbf{1}/n)$ denotes a divergence (either Kullback-Leibler or $\chi^2$) between a distribution on $n$ atoms and the uniform distribution $\mathbf{1}/n = (1/n, \ldots, 1/n)$,
- $\nu \geq 0$ is a dual regularization parameter, or the "shift cost",

The set $\mathcal{Q}$ is also a hyperparameter of the method, but is often indexed by a single univariate quantity, as we describe below.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from drlearn import make_extremile_spectrum, Ridge, BinaryLogisticRegression, MultinomialLogisticRegression

While one may specify $\nu$ and the divergence measure, these are set to defaults that generally do not need to change. However, the user might wish to adjust $\mathcal{Q}$, which is of the form

$$
    \mathcal{Q} \equiv \mathcal{Q}(\sigma) = \operatorname{conv}\{\text{permutations of $\sigma$}\},
$$

where $\sigma = (\sigma_1, \ldots, \sigma_n)$ is a vector of non-negative weights that sums to one, called the *spectrum*. We call $\mathcal{Q}(\sigma)$ is the *permutahedron* associated to the vector $\sigma$. Various choices of $\sigma$ can be generated by using the `make_<spectrum_name>_spectrum` functions within the package, which return Numpy arrays with value equal to $\sigma$. See "Setting Risk Parameters" in the [documentation site](https://ronakdm.github.io/drlearn/) for more extensive information on spectral risk measures and how to set $\mathcal{Q}(\sigma)$. After specifying the spectrum, one can use estimators in the `scikit-learn` interface to fit linear and logit-linear models on data. Examples are shown below.

**Regression:** This can be performed using `Ridge`, which mirrors the `Ridge` object from `sklearn.linear_model`. This can be applied to datasets with numerical values, which is a major motivation of distributionally robust methods, as the loss (such as the squared error) can be the ultimate metric of interest.

In [6]:
X, y = fetch_openml(name="yacht_hydrodynamics", return_X_y=True, data_home="data/", as_frame=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# prepare data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
center, spread = np.mean(y_train), np.std(y_train)
y_train = (y_train - center) / spread
y_test = (y_test - center) / spread

(246, 6)
(246,)
(62, 6)
(62,)


In [7]:
n, d = X_train.shape
spectrum = make_extremile_spectrum(n, 1.5)
model1 = Ridge(spectrum=np.ones(shape=(n,)) / n, fit_intercept=False).fit(X_train, y_train)
model2 = Ridge(spectrum=spectrum, fit_intercept=False).fit(X_train, y_train)

`Ridge` objects have standard `predict` methods.

In [8]:
loss1 = (y_test - model1.predict(X_test))**2
loss2 = (y_test - model2.predict(X_test))**2

print(f"Test loss 0.5-quantile of model1: {np.quantile(loss1, 0.5):.4f}")
print(f"Test loss 0.5-quantile of model2: {np.quantile(loss2, 0.5):.4f}")

print(f"Test loss 0.8-quantile of model1: {np.quantile(loss1, 0.8):.4f}")
print(f"Test loss 0.8-quantile of model2: {np.quantile(loss2, 0.8):.4f}")

Test loss 0.5-quantile of model1: 0.1163
Test loss 0.5-quantile of model2: 0.0944
Test loss 0.8-quantile of model1: 0.2399
Test loss 0.8-quantile of model2: 0.2378


**Classification:** This task can be performed using either the `BinaryLogisticRegression` or the `MultinomialLogisticRegression` objects, depending on whether the problem is framed as a binary or multiclass classification problem. We demonstrate both on a binary classification problem.

In [9]:
X, y = fetch_openml(name="blood-transfusion-service-center", version=1, return_X_y=True, data_home="data/", as_frame=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# prepare data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# binary classification with labels in {0, 1}
y_train = y_train.astype(int) - 1
y_test = y_test.astype(int) - 1

(598, 4)
(598,)
(150, 4)
(150,)


In [10]:
n, d = X_train.shape
spectrum = make_extremile_spectrum(n, 1.5)
model1 = BinaryLogisticRegression(spectrum=spectrum).fit(X_train, y_train)
model2 = MultinomialLogisticRegression(spectrum=spectrum).fit(X_train, y_train)

For `BinaryLogisticRegression`, the predicted probabilities will refer to the "positive" class (class 1 and *not* class 0). For `MultinomialLogisticRegression`, the return value will be of size `(input_len, n_classes)`. 

In [11]:
probas_pred_binary = model1.predict_proba(X_test)
probas_pred_multinomial = model2.predict_proba(X_test)

print(probas_pred_binary.shape)
print(probas_pred_binary[:5])

print(probas_pred_multinomial.shape)
print(probas_pred_multinomial[:5])

(150,)
[0.33138687 0.31341856 0.33675978 0.3160968  0.30987164]
(150, 2)
[[0.66703463 0.33296537]
 [0.6989987  0.3010013 ]
 [0.66149036 0.33850964]
 [0.69224    0.30776   ]
 [0.70225582 0.29774418]]
